In [ ]:
from redpitaya.overlay.mercury import mercury as overlay
fpga = overlay()

gen0 = fpga.gen(0)
osc0 = fpga.osc(0, 1.0)
mgmt = fpga.mgmt()

In [ ]:
# buffer waveform and sample timing
gen0.waveform      = [1,0]

# burst half the buffer with then idle for quarter buffer, repeat 4 times
gen0.burst_data_repetitions = 1
gen0.burst_data_length   =  gen0.buffer_size // 2
gen0.burst_period_length = (gen0.buffer_size // 4) * 3
gen0.burst_period_number = 4
gen0.mode              = 'FINITE'

# set output amplitude, offset and enable it
gen0.amplitude     = 1.0
gen0.offset        = 0
gen0.enable        = True

In [ ]:
# data rate decimation 
osc0.decimation = 1

# trigger timing
N = 32
osc0.trigger_pre  = 0
osc0.trigger_post = 32

# synchronization signals from 
osc0.sync_src = fpga.sync_src['gen0']
osc0.trig_src = 0

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure, output_file, show
from bokeh.models import LabelSet, Label
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

# plotting
p = figure()
p.y_range=Range1d(-1, 1)
r = p.line(range(N), [0]*N)
# get and explicit handle to update the next show cell 
target = show(p,notebook_handle=True)

In [ ]:
# digital loopback
mgmt.loop = 0b11

while True:
    # reset, start and trigger generator to get the first burst
    gen0.reset()
    gen0.start()
    gen0.trigger()
    while (osc0.status_run()): pass
    r.data_source.data['y'] = osc0.data(N)
    push_notebook(handle=target)